In [20]:
from langchain_groq import ChatGroq

In [21]:
from dotenv import load_dotenv
import os
load_dotenv()
llm = ChatGroq(
    temperature=0,
    groq_api_key=os.getenv('GROQ_API_KEY'),
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [22]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/80548679106077382-software-engineer-ii-full-stack-youtube-channel-memberships")
page_data = loader.load().pop().page_content
print(page_data)

Software Engineer II, Full Stack, YouTube Channel Memberships — Google CareersCareersCareersSkip navigation linkshomehomeHomeHomework_outlinework_outlineJobsJobsnoogler_hatnoogler_hatStudentsStudentsgooglegoogleHow we workHow we workhandymanhandymanHow we hireHow we hireperson_outlineperson_outlineYour careerYour careerhelp_outlineHelp linkfeedbackSend feedbackmore_vert HelpSend FeedbackSign inCareersCareershomeHomework_outlineJobsexpand_morenoogler_hatStudentsexpand_moregoogleHow we workexpand_morehandymanHow we hireexpand_moreperson_outlineYour careerexpand_morejob detailsarrow_backBack to jobs searchJobs search results2,480  jobs matchedSolutions Manager, AI, Public SectorReston, VA, USA; Atlanta, GA, USA; +4 more; +3 moreSenior Financial Analyst, Technical Infrastructure (Mandarin, English)Taipei, TaiwanSoftware Engineer II, Full Stack, YouTube Channel MembershipsBengaluru, Karnataka, IndiaQuản lý Khách hàng Chiến lược cấp cao, ngành Hàng tiêu dùng đóng gói sẵn, khối Khách hàng chi

In [23]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [24]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer II, Full Stack, YouTube Channel Memberships',
  'experience': '1 year of experience with full stack development',
  'skills': ['Java',
   'Python',
   'GO',
   'C++',
   'JavaScript',
   'TypeScript',
   'HTML',
   'CSS',
   'data structures',
   'algorithms'],
  'description': 'Write product or system development code, review code developed by other developers, contribute to existing documentation, triage product or system issues and debug/track/resolve.'},
 {'role': 'Solutions Manager, AI, Public Sector',
  'experience': 'Not specified',
  'skills': ['AI', 'Public Sector'],
  'description': 'Not specified'},
 {'role': 'Senior Financial Analyst, Technical Infrastructure (Mandarin, English)',
  'experience': 'Not specified',
  'skills': ['Mandarin', 'English', 'Financial analysis'],
  'description': 'Not specified'},
 {'role': 'Quản lý Khách hàng Chiến lược cấp cao, ngành Hàng tiêu dùng đóng gói sẵn, khối Khách hàng chiến lược (tiếng Anh, tiếng Việt)',
  'e

In [25]:
type(json_res)

list

In [26]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [27]:
import uuid
import chromadb
from chromadb.config import Settings

client = chromadb.PersistentClient('vectorstore', settings=Settings(anonymized_telemetry=False))
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [29]:
job = json_res[0]
job['skills']

['Java',
 'Python',
 'GO',
 'C++',
 'JavaScript',
 'TypeScript',
 'HTML',
 'CSS',
 'data structures',
 'algorithms']

In [30]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com

In [31]:
from langchain_core.prompts import PromptTemplate

prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Priyanshu Sekhar Panda, final year B.Tech student at NIT Rourkela, applying for a job role mentioned above. 
        Your job is to write a professional cold email addressed to the hiring manager regarding this opportunity.  
        The email should introduce yourself, highlight your skills and experience relevant to the role, and explain why you are a strong fit.  
        Also mention that you have attached your resume for their consideration. {link_list}  
        Do not provide a preamble.  

        ### EMAIL (NO PREAMBLE):

        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Software Engineer II, Full Stack, YouTube Channel Memberships

Dear Hiring Manager,

I am Priyanshu Sekhar Panda, a final year B.Tech student at NIT Rourkela, and I am excited to apply for the Software Engineer II, Full Stack, YouTube Channel Memberships role at your esteemed organization. With a strong foundation in computer science and a passion for full stack development, I am confident that I would be a valuable addition to your team.

As a skilled programmer with experience in languages such as Java, Python, GO, C++, JavaScript, and TypeScript, I possess a solid understanding of data structures and algorithms. My proficiency in HTML and CSS has enabled me to develop visually appealing and user-friendly interfaces. Throughout my academic journey, I have worked on various projects that demonstrate my ability to write efficient code, review code developed by others, and contribute to existing documentation.

I am particularly drawn to this role because of the